# Test Hot Pixel Groups

This notebook tests an algorithm that finds hot pixels, and organizes them into hot pixel groups.

## Create the Test Data

In [1]:
# THIS COMMENT IS THE LONGEST A LINE CAN BE AND STILL RENDER COMPLETELY WHEN PRINTING IN LANDSCAPE MODE.

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from collections import namedtuple

HotPixel = namedtuple('HotPixel', 'x y value')
HotPixelGroup = namedtuple('HotPixelGroup', 'leader_index hot_pixels')

TEST_DATA_WIDTH = 6
TEST_DATA_HEIGHT = 4

TEST_DATA = np.zeros((TEST_DATA_HEIGHT, TEST_DATA_WIDTH))

TEST_HOT_PIXEL_GROUPS = [
    HotPixelGroup(1, [
        HotPixel(0, 0, 275), 
        HotPixel(0, 1, 300), # Group leader
        HotPixel(1, 1, 250)
    ]),
    HotPixelGroup(1, [
        HotPixel(3, 0, 275),
        HotPixel(4, 1, 275), # Group leader that tests tie-breaking algorithm
        HotPixel(3, 2, 225),
        HotPixel(4, 2, 250),
        HotPixel(5, 2, 225),
        HotPixel(5, 3, 325)
    ]),
    HotPixelGroup(0, [
        HotPixel(1, 3, 250)  # Isolated hot pixel
    ]),
    HotPixelGroup(5, [
        HotPixel(3, 0, 275), # Note that this group of six pixels is the same as one of the above
        HotPixel(4, 1, 275),
        HotPixel(3, 2, 225),
        HotPixel(4, 2, 250),
        HotPixel(5, 2, 225),
        HotPixel(5, 3, 325)  # Also a group leader
    ])   
]

for hot_pixel_group in TEST_HOT_PIXEL_GROUPS:
    hot_pixels = hot_pixel_group.hot_pixels
    for hot_pixel in hot_pixels:
        TEST_DATA[hot_pixel.y, hot_pixel.x] = hot_pixel.value

TEST_DATA

array([[275.,   0.,   0., 275.,   0.,   0.],
       [300., 250.,   0.,   0., 275.,   0.],
       [  0.,   0.,   0., 225., 250., 225.],
       [  0., 250.,   0.,   0.,   0., 325.]])

## Locate all Hot Pixel Groups in the Test Data


As a first cut, we will search for all pixels that have more than 200 ADU after subtracting the master dark.

In [2]:
print("Values bigger than 200 =", TEST_DATA[TEST_DATA > 200])

Values bigger than 200 = [275. 275. 300. 250. 275. 225. 250. 225. 250. 325.]


In [3]:
exceedances = TEST_DATA > 200  # an array of true-false values

values_of_exceedances = TEST_DATA[exceedances]

exceedance_indices = np.nonzero(exceedances)  # a crafty way of getting the indices of the exceedances

# zip all of what we have found together

hot_pixel_list = np.transpose([exceedance_indices[1], exceedance_indices[0], values_of_exceedances])

hot_pixel_list

array([[  0.,   0., 275.],
       [  3.,   0., 275.],
       [  0.,   1., 300.],
       [  1.,   1., 250.],
       [  4.,   1., 275.],
       [  3.,   2., 225.],
       [  4.,   2., 250.],
       [  5.,   2., 225.],
       [  1.,   3., 250.],
       [  5.,   3., 325.]])

## Find Contiguous Hot Pixels and Pixel Group Leaders

Find the contiguous groups of hot pixels.

Each hot pixel is examined to see if it is the brightest relative to the eight nearest neighbors. If it is, the brightest of its eight neighbors, then it is added to the leaders. If two neighbors have exactly the same value, the one with the more even indices wins.

It becomes the leader of a pixel group containing that contains all contiguous hot pixels

In [4]:
def is_winner(candidate_leader, i, j, data):
    if candidate_leader[2] > data[j, i]:
        return True
    elif candidate_leader[2] == data[j, i]:
        # some nasty tie-breaking
        if candidate_leader[0] > i:
            return True
        elif candidate_leader[0] == i and candidate_leader[1] > j:
            return True        

def is_leader(candidate_leader, data):
    data_height, data_width = data.shape
    for j in [-1, 0, 1]:
        if j < 0 or j >= data_height:
            continue
        for i in [-1, 0, 1]:
            if i < 0 or i >= data_width:
                continue
            if not is_winner(candidate_leader, i, j, data):
                return False
    return True

def find_hot_pixel_groups(data, threshold=200):
    data_height, data_width = data.shape
    exceedances = data > threshold  # an array of true-false values
    values_of_exceedances = data[exceedances]
    exceedance_indices = np.nonzero(exceedances)  # a crafty way of getting the indices of the exceedances
    candidate_leaders = np.transpose([exceedance_indices[1], exceedance_indices[0], values_of_exceedances])
    for i in range(candidate_leaders.shape[0]):
        print(candidate_leaders[i])
        print(i, ' ', is_leader(candidate_leaders[i], data))

#   if (is_leader(hot_pixel, representative_dark_data)):
#        hot_pixel_leaders.append(hot_pixel)

find_hot_pixel_groups(TEST_DATA, 200)
# np.set_printoptions(threshold=sys.maxsize) # Uncommenting this line will cause serious I/O strain


[  0.   0. 275.]
0   False
[  3.   0. 275.]
1   False
[  0.   1. 300.]
2   False
[  1.   1. 250.]
3   False
[  4.   1. 275.]
4   False
[  3.   2. 225.]
5   False
[  4.   2. 250.]
6   False
[  5.   2. 225.]
7   False
[  1.   3. 250.]
8   False
[  5.   3. 325.]
9   True
